<a href="https://colab.research.google.com/github/ThensiB/RepVGG-GELAN/blob/main/yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
!nvidia-smi

Sun Apr 14 00:15:06 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8              11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
!pip install -r /content/drive/MyDrive/RGELAN/requirements.txt -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.9 MB/s eta 0:00:00


# k cross validation

In [ ]:
import os
import yaml
import shutil

# Define paths
yolov9_path = '/content/drive/MyDrive/yolov9'  # Update with the path to YOLOv9 directory
custom_yaml_path = '/content/drive/MyDrive/yolov9/data/coco.yaml'  # Update with the path to custom.yaml
data_path = '/content/drive/MyDrive/Folds'


In [ ]:
# Function to create training folders for each fold
def create_train_folders(train_folders, n):
    for idx, folder in enumerate(train_folders):
        train_folder = f'/content/drive/MyDrive/Folds/train{n}'
        os.makedirs(os.path.join(train_folder, 'images'), exist_ok=True)
        os.makedirs(os.path.join(train_folder, 'labels'), exist_ok=True)

        # Copy images and labels from original training folders
        images_src = os.path.join(folder, 'images')
        labels_src = os.path.join(folder, 'labels')

        print(labels_src)

        images_dst = os.path.join(train_folder, 'images')
        labels_dst = os.path.join(train_folder, 'labels')

        print(images_dst)

        for image_file in os.listdir(images_src):
            shutil.copy(os.path.join(images_src, image_file), images_dst)

        for label_file in os.listdir(labels_src):
            shutil.copy(os.path.join(labels_src, label_file), labels_dst)

        # Count images and labels
        num_images = len(os.listdir(images_dst))
        num_labels = len(os.listdir(labels_dst))
    print(f"Train{n} - Images: {num_images}, Labels: {num_labels}")

    return train_folder


In [ ]:
import shutil
p = '/content/drive/MyDrive/Folds/train1'
shutil.rmtree(p)

In [ ]:
# Function to train YOLOv9
def train_yolov9(train_folders, val_folder, fold_idx):
    # Load custom.yaml
    with open(custom_yaml_path, 'r') as file:
        yaml_content = yaml.safe_load(file)

    # Update train and val paths
    yaml_content['path'] = data_path
    yaml_content['train'] = train_folders
    yaml_content['val'] = val_folder

    print(yaml_content['val'])

    # Write the updated YAML content back to the file
    with open(custom_yaml_path, 'w') as file:
        yaml.dump(yaml_content, file)

    !python {yolov9_path}/train.py --img 640 --batch 8 --epochs 150 --min-items 0 --data {custom_yaml_path} --cfg {yolov9_path}/models/detect/rcs-gelan-c.yaml --hyp {yolov9_path}/data/hyps/hyp.scratch-high.yaml --device 0 --name fold{fold_idx}


# Get list of fold directories
fold_directories = [f'/content/drive/MyDrive/Folds/fold_{i+1}' for i in range(5)]

# Iterate over each fold for validation
for fold_idx, val_folder in enumerate(fold_directories):

    if fold_idx == 4:
      print(f"Training on fold {fold_idx + 1}...")

      # Define training folders (all other folds)
      train_folders = fold_directories[:fold_idx] + fold_directories[fold_idx + 1:]

      # Create training folders for the current fold
      train_fold = create_train_folders(train_folders, fold_idx + 1)
      print(train_fold)

      # Train YOLOv9 on the current fold
      train_yolov9(train_fold, val_folder, fold_idx + 1)



Training on fold 5...
/content/drive/MyDrive/Folds/fold_1/labels
/content/drive/MyDrive/Folds/train5/images
/content/drive/MyDrive/Folds/fold_2/labels
/content/drive/MyDrive/Folds/train5/images
/content/drive/MyDrive/Folds/fold_3/labels
/content/drive/MyDrive/Folds/train5/images
/content/drive/MyDrive/Folds/fold_4/labels
/content/drive/MyDrive/Folds/train5/images
Train5 - Images: 561, Labels: 561
/content/drive/MyDrive/Folds/train5
/content/drive/MyDrive/Folds/fold_5
2024-04-01 11:32:08.050454: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-01 11:32:08.050508: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-01 11:32:08.051887: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to regi

In [ ]:
#RCS-GELAN

# Function to train YOLOv9
def train_yolov9(train_folders, val_folder, fold_idx):
    # Load custom.yaml
    with open(custom_yaml_path, 'r') as file:
        yaml_content = yaml.safe_load(file)

    # Update train and val paths
    yaml_content['path'] = data_path
    yaml_content['train'] = train_folders
    yaml_content['val'] = val_folder

    print(yaml_content['val'])

    # Write the updated YAML content back to the file
    with open(custom_yaml_path, 'w') as file:
        yaml.dump(yaml_content, file)

    !python {yolov9_path}/train.py --img 640 --batch 8 --epochs 150 --min-items 0 --close-mosaic 15 --data {custom_yaml_path} --cfg {yolov9_path}/models/detect/rcs-gelan-c.yaml --hyp {yolov9_path}/data/hyps/hyp.scratch-high.yaml --device 0 --name fold{fold_idx}


# Get list of fold directories
fold_directories = [f'/content/drive/MyDrive/Folds/fold_{i+1}' for i in range(5)]

# Iterate over each fold for validation
for fold_idx, val_folder in enumerate(fold_directories):
    print(f"Training on fold {fold_idx + 1}...")

    # Define training folders (all other folds)
    train_folders = fold_directories[:fold_idx] + fold_directories[fold_idx + 1:]

    # Create training folders for the current fold
    train_fold = create_train_folders(train_folders, fold_idx + 1)
    print(train_fold)

    # Train YOLOv9 on the current fold
    train_yolov9(train_fold, val_folder, fold_idx + 1)

# After training, you can aggregate and analyze the results as needed


Training on fold 1...
/content/drive/MyDrive/Folds/fold_2/labels
/content/drive/MyDrive/Folds/train1/images
/content/drive/MyDrive/Folds/fold_3/labels
/content/drive/MyDrive/Folds/train1/images
/content/drive/MyDrive/Folds/fold_4/labels
/content/drive/MyDrive/Folds/train1/images
/content/drive/MyDrive/Folds/fold_5/labels
/content/drive/MyDrive/Folds/train1/images
Train1 - Images: 560, Labels: 560
/content/drive/MyDrive/Folds/train1
/content/drive/MyDrive/Folds/fold_1
2024-03-28 01:25:27.874895: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-28 01:25:27.874946: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-28 01:25:27.882947: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to regi

# training on brain dataset

In [ ]:
!python /content/drive/MyDrive/RGELAN/train.py \
--batch 8 --epochs 150 --img 640 --device 0 --min-items 0 --close-mosaic 15 \
--data /content/drive/MyDrive/RGELAN/data/coco.yaml \
--cfg /content/drive/MyDrive/RGELAN/models/detect/rcs-gelan-c.yaml \
--hyp /content/drive/MyDrive/RGELAN/data/hyps/hyp.scratch-high.yaml

2024-04-08 23:33:02.180845: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-08 23:33:02.180900: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-08 23:33:02.224710: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-08 23:33:03.347296: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
train: weights=, cfg=/content/drive/MyDrive/RGELAN/models/detect/rcs-gelan-c.yaml, data=/content/drive/MyDrive/RGELAN/data/coco.yaml, hyp=/content/drive/MyDrive/RGELAN/data/hyps/hyp.scratch-high.yaml, epochs=150, batch_size=8, imgsz=640, rec

In [ ]:
!python /content/drive/MyDrive/RGELAN/train.py \
--batch 8 --epochs 150 --img 640 --device 0 --min-items 0 --close-mosaic 15 \
--data /content/drive/MyDrive/RGELAN/data/coco.yaml \
--cfg /content/drive/MyDrive/RGELAN/models/detect/rcs-gelan-c.yaml \
--hyp /content/drive/MyDrive/RGELAN/data/hyps/hyp.scratch-high.yaml

2024-04-09 12:03:58.553444: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-09 12:03:58.553500: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-09 12:03:58.558995: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-09 12:03:59.703030: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
train: weights=, cfg=/content/drive/MyDrive/RGELAN/models/detect/rcs-gelan-c.yaml, data=/content/drive/MyDrive/RGELAN/data/coco.yaml, hyp=/content/drive/MyDrive/RGELAN/data/hyps/hyp.scratch-high.yaml, epochs=150, batch_size=8, imgsz=640, rec

In [ ]:
!python /content/drive/MyDrive/RGELAN/train.py \
--batch 8 --epochs 150 --img 640 --device 0 --min-items 0 --close-mosaic 15 \
--data /content/drive/MyDrive/RGELAN/data/coco.yaml \
--cfg /content/drive/MyDrive/RGELAN/models/detect/gelan-c.yaml \
--hyp /content/drive/MyDrive/RGELAN/data/hyps/hyp.scratch-high.yaml

2024-04-09 08:55:09.013619: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-09 08:55:09.013674: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-09 08:55:09.146450: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-09 08:55:10.221013: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
train: weights=, cfg=/content/drive/MyDrive/RGELAN/models/detect/gelan-c.yaml, data=/content/drive/MyDrive/RGELAN/data/coco.yaml, hyp=/content/drive/MyDrive/RGELAN/data/hyps/hyp.scratch-high.yaml, epochs=150, batch_size=8, imgsz=640, rect=Fa

In [ ]:
!python /content/drive/MyDrive/RGELAN/train.py \
--batch 8 --epochs 150 --img 640 --device 0 --min-items 0 --close-mosaic 15 \
--data /content/drive/MyDrive/RGELAN/data/coco.yaml \
--cfg /content/drive/MyDrive/RGELAN/models/detect/gelan-c.yaml \
--hyp /content/drive/MyDrive/RGELAN/data/hyps/hyp.scratch-high.yaml

2024-04-09 10:32:25.170334: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-09 10:32:25.170385: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-09 10:32:25.171964: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-09 10:32:26.385080: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
train: weights=, cfg=/content/drive/MyDrive/RGELAN/models/detect/gelan-c.yaml, data=/content/drive/MyDrive/RGELAN/data/coco.yaml, hyp=/content/drive/MyDrive/RGELAN/data/hyps/hyp.scratch-high.yaml, epochs=150, batch_size=8, imgsz=640, rect=Fa

# yolov8 on brain

In [ ]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.4/750.4 kB 9.0 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO

from IPython.display import display, Image

In [ ]:
import os
import shutil
import random

!pip install tqdm --upgrade
from tqdm.notebook import tqdm

In [ ]:
ROOT_DIR = '/content/drive/MyDrive'

In [ ]:
# Load a model
model = YOLO("yolov8n.pt")  # load pre trained model

# Use the model
results = model.train(data=os.path.join(ROOT_DIR, "coco.yaml"), epochs=150)  # train the model


Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/drive/MyDrive/coco.yaml, epochs=150, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=Tru

train: Scanning /content/drive/MyDrive/brain/TRAIN/labels... 500 images, 3 backgrounds, 0 corrupt: 100%|██████████| 500/500 [06:43<00:00,  1.24it/s]


train: New cache created: /content/drive/MyDrive/brain/TRAIN/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/drive/MyDrive/brain/VAL/labels... 201 images, 0 backgrounds, 0 corrupt: 100%|██████████| 201/201 [02:37<00:00,  1.27it/s]

val: New cache created: /content/drive/MyDrive/brain/VAL/labels.cache


...

    150/150      2.29G     0.5286       0.29     0.8902          5        640: 100%|██████████| 32/32 [00:11<00:00,  2.70it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:02<00:00,  2.57it/s]

                   all        201        240      0.962      0.921      0.954      0.712



150 epochs completed in 0.677 hours.
Optimizer stripped from runs/detect/train3/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train3/weights/best.pt, 6.2MB

Validating runs/detect/train3/weights/best.pt...
Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:05<00:00,  1.22it/s]


                   all        201        240      0.973      0.909      0.957      0.733
Speed: 0.6ms preprocess, 3.4ms inference, 0.0ms loss, 4.2ms postprocess per image
Results saved to runs/detect/train3


In [ ]:
!python /content/drive/MyDrive/RGELAN/train.py \
--batch 8 --epochs 150 --img 640 --device 0 --min-items 0 --close-mosaic 15 \
--data /content/drive/MyDrive/RGELAN/data/coco2.yaml \
--cfg /content/drive/MyDrive/RGELAN/models/detect/rcs-gelan-c.yaml \
--hyp /content/drive/MyDrive/RGELAN/data/hyps/hyp.scratch-high.yaml

2024-04-14 00:16:38.760211: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-14 00:16:38.760269: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-14 00:16:38.761723: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-14 00:16:39.884999: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
train: weights=, cfg=/content/drive/MyDrive/RGELAN/models/detect/rcs-gelan-c.yaml, data=/content/drive/MyDrive/RGELAN/data/coco2.yaml, hyp=/content/drive/MyDrive/RGELAN/data/hyps/hyp.scratch-high.yaml, epochs=150, batch_size=8, imgsz=640, re